# Field operations

There are several convenience methods that can be used to analyse the field. Let us first define the mesh we are going to work with.

In [1]:
import discretisedfield as df

p1 = (-50, -50, -50)
p2 = (50, 50, 50)
n = (2, 2, 2)
mesh = df.Mesh(p1=p1, p2=p2, n=n)

We are going to initialise the vector field (`dim=3`), with

$$\mathbf{f}(x, y, z) = (xy, 2xy, xyz)$$

For that, we are going to use the following Python function.

In [2]:
def value_function(pos):
    x, y, z = pos
    return x*y, 2*x*y, x*y*z

Finally, our field is

In [3]:
field = df.Field(mesh, dim=3, value=value_function)

## 1. Sampling the field

As we have shown previously, a field can be sampled by calling it. The argument must be a 3-length iterable and it contains the coordinates of the point.

In [4]:
point = (0, 0, 0)
field(point)

(625.0, 1250.0, -15625.0)

However if the point is outside the mesh, an exception is raised.

In [5]:
point = (100, 100, 100)
try:
    field(point)
except ValueError:
    print('Exception raised.')

Exception raised.


## 2. Extracting the component of a vector field

A three-dimensional vector field can be understood as three separate scalar fields, where each scalar field is a component of a vector field value. A scalar field of a component can be extracted by accessing `x`, `y`, or `z` attribute of the field.

In [6]:
x_component = field.x
x_component((0, 0, 0))

625.0

Default names `x`, `y`, and (for dim 3) `z` are only available for fields with dimensionality 2 or 3.

In [7]:
field.components

['x', 'y', 'z']

It is possible to change the component names:

In [8]:
field.components = ['mx', 'my', 'mz']
field.mx((0, 0, 0))

625.0

This overrides the component labels and the old `x`, `y` and `z` cannot be used anymore:

In [9]:
try:
    field.x
except AttributeError as e:
    print(e)

Object has no attribute x.


We change the component labels back to `x`, `y`, and `z` for the rest of this notebook.

In [10]:
field.components = ['x', 'y', 'z']

Custom component names can optionally also be specified during field creation. If not specified, the default values are used for fields with dimensions 2 or 3. Higher-dimensional fields have no defaults and custom labes have to be specified in order to access individual field components:

In [11]:
field_4d = df.Field(mesh, dim=4, value=[1, 1, 1, 1], components=['c1', 'c2', 'c3', 'c4'])
field_4d

Field(mesh=Mesh(region=Region(p1=(-50, -50, -50), p2=(50, 50, 50)), n=(2, 2, 2), bc='', subregions={}, attributes={'unit': 'm', 'fourierspace': False, 'isplane': False}), dim=4, components=['c1', 'c2', 'c3', 'c4'])

In [12]:
field_4d.c1((0, 0, 0))

1.0

## 3. Computing the average

The average of the field can be obtained by calling `discretisedfield.Field.average` property.

In [13]:
field.average

(0.0, 0.0, 0.0)

Average always return a tuple, independent of the dimension of the field's value.

In [14]:
field.x.average

0.0

## 4. Iterating through the field

The field object itself is an iterable. That means that it can be iterated through. As a result it returns a tuple, where the first element is the coordinate of the mesh point, whereas the second one is its value.

In [15]:
for coordinate, value in field:
    print(coordinate, value)

(-25.0, -25.0, -25.0) (625.0, 1250.0, -15625.0)
(25.0, -25.0, -25.0) (-625.0, -1250.0, 15625.0)
(-25.0, 25.0, -25.0) (-625.0, -1250.0, 15625.0)
(25.0, 25.0, -25.0) (625.0, 1250.0, -15625.0)
(-25.0, -25.0, 25.0) (625.0, 1250.0, 15625.0)
(25.0, -25.0, 25.0) (-625.0, -1250.0, -15625.0)
(-25.0, 25.0, 25.0) (-625.0, -1250.0, -15625.0)
(25.0, 25.0, 25.0) (625.0, 1250.0, 15625.0)


## 5. Sampling the field along the line

To sample the points of the field which are on a certain line, `discretisedfield.Field.line` method is used. It takes two points `p1` and `p2` that define the line and an integer `n` which defines how many mesh coordinates on that line are required. The default value of `n` is 100.

In [16]:
line = field.line(p1=(-10, 0, 0), p2=(10, 0, 0), n=5)

## 6. Intersecting the field with a plane

If we intersect the field with a plane, `discretisedfield.Field.plane` will return a new field object which contains only discretisation cells that belong to that plane. The planes allowed are the planes perpendicular to the axes of the Cartesian coordinate system. For instance, a plane parallel to the $yz$-plane (perpendicular to the $x$-axis) which intesects the $x$-axis at 1, can be written as

$$x = 1$$

In [17]:
field.plane(x=1)

Field(mesh=Mesh(region=Region(p1=(-24.0, -50.0, -50.0), p2=(26.0, 50.0, 50.0)), n=(1, 2, 2), bc='', subregions={}, attributes={'unit': 'm', 'fourierspace': False, 'isplane': True, 'planeaxis': 0, 'point': 1, 'axis1': 1, 'axis2': 2}), dim=3, components=['x', 'y', 'z'])

If we want to cut through the middle of the mesh, we do not need to provide a particular value for a coordinate.

In [18]:
field.plane('x')

Field(mesh=Mesh(region=Region(p1=(-25.0, -50.0, -50.0), p2=(25.0, 50.0, 50.0)), n=(1, 2, 2), bc='', subregions={}, attributes={'unit': 'm', 'fourierspace': False, 'isplane': True, 'planeaxis': 0, 'point': 0.0, 'axis1': 1, 'axis2': 2}), dim=3, components=['x', 'y', 'z'])

## 7. Cascading the operations

Let us say we want to compute the average of an $x$ component of the field on the plane $y=10$. In order to do that, we can cascade several operation in a single line.

In [19]:
field.plane(y=10).x.average

0.0

This gives the same result as for instance

In [20]:
field.x.plane(y=10).average

0.0

## Other

Full description of all existing functionality can be found in the [API Reference](https://discretisedfield.readthedocs.io/en/latest/_autosummary/discretisedfield.Field.html).